# Neural Machine Translation

In this work you need to implement any Seq2seq architecture to train neural German-Ukrainian translator.

Mostly copy of [practical-pytorch](https://github.com/spro/practical-pytorch/blob/master/seq2seq-translation/seq2seq-translation.ipynb) tutorial.

In [146]:
import string
import re
import random
import time
import math
import os

import pandas as pd
import nltk
from tokenize_uk import tokenize_words
from tqdm import tqdm_notebook as tqdm

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

Managing **environment** (CPU or GPU) is not simple in pytorch. You need to explicitly select environment for all tensors in your model. Use this constant to define GPU usage.

In [2]:
USE_CUDA = False

## Load data

We will use open source sentence pairs from [tatoeba.org](https://tatoeba.org/eng/downloads). Download [sentences archive](http://downloads.tatoeba.org/exports/sentences.tar.bz2) and extract it into `./data/part3`.

In [3]:
%env DATA_DIR = ./../data/part3/

env: DATA_DIR=./../data/part3/


In [4]:
!wget https://downloads.tatoeba.org/exports/sentences.tar.bz2 -P $DATA_DIR
!tar xvjC $DATA_DIR -f $DATA_DIR/sentences.tar.bz2

--2018-07-07 14:20:50--  https://downloads.tatoeba.org/exports/sentences.tar.bz2
Resolving downloads.tatoeba.org (downloads.tatoeba.org)... 94.130.77.194
Connecting to downloads.tatoeba.org (downloads.tatoeba.org)|94.130.77.194|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 106890169 (102M) [application/octet-stream]
Saving to: ‘./../data/part3/sentences.tar.bz2.1’

sentences.tar.bz2.1 100%[===================>] 101.94M  2.20MB/s    in 47s     

2018-07-07 14:21:37 (2.19 MB/s) - ‘./../data/part3/sentences.tar.bz2.1’ saved [106890169/106890169]

x sentences.csv


In [5]:
!wget https://downloads.tatoeba.org/exports/links.tar.bz2 -P $DATA_DIR
!tar xvjC $DATA_DIR -f $DATA_DIR/links.tar.bz2

--2018-07-07 14:21:54--  https://downloads.tatoeba.org/exports/links.tar.bz2
Resolving downloads.tatoeba.org (downloads.tatoeba.org)... 94.130.77.194
Connecting to downloads.tatoeba.org (downloads.tatoeba.org)|94.130.77.194|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 72991002 (70M) [application/octet-stream]
Saving to: ‘./../data/part3/links.tar.bz2.1’

links.tar.bz2.1     100%[===================>]  69.61M  2.21MB/s    in 32s     

2018-07-07 14:22:26 (2.19 MB/s) - ‘./../data/part3/links.tar.bz2.1’ saved [72991002/72991002]

x links.csv


In [6]:
data_dir = './../data/part3/'
sentences = pd.read_csv(os.path.join(data_dir, 'sentences.csv'), names=['id', 'lang', 'text'], header=None, delimiter='\t')
links = pd.read_csv(os.path.join(data_dir, 'links.csv'), names=['sent_id', 'tran_id'], header=None, delimiter='\t')

Choose any languages you want to train translator for.

In [8]:
source_lang = 'ukr'
target_lang = 'deu'

source_sentences = sentences[sentences.lang == source_lang]
source_sentences = source_sentences.merge(links, left_on='id', right_on='sent_id')
target_sentences = sentences[sentences.lang == target_lang]

bilang_sentences = source_sentences.merge(target_sentences, left_on='tran_id', 
                                          right_on='id', 
                                          suffixes=[source_lang, target_lang])
bilang_sentences = bilang_sentences[['text'+source_lang, 'text'+target_lang]]

file_name = os.path.join(data_dir, '{source}-{target}.csv'.format(source=source_lang, target=target_lang)) 
bilang_sentences.to_csv(file_name, index=False, sep='\t')

bilang_sentences.head()

,textukr,textdeu
0,Він наказав мені негайно вийти з кімнати.,"Er befahl mir, den Raum umgehend zu verlassen."
1,У всесвіті багато галактик.,Es gibt viele Galaxien im Universum.
2,У Всесвіті є багато галактик.,Es gibt viele Galaxien im Universum.
3,Вона приймає душ щоранку.,Sie nimmt jeden Morgen eine Dusche.
4,Вона приймає душ щоранку.,Sie duscht jeden Morgen.


In [9]:
len(bilang_sentences)

14755

### Indexing words

We'll need a unique index per word to use as the inputs and targets of the networks later. To keep track of all this we will use a helper class called Lang which has word → index (word2index) and index → word (index2word) dictionaries, as well as a count of each word word2count to use to later replace rare words.

In [165]:
SOS_token = 0
EOS_token = 1
UNK_token = 2
PAD_token = 3

class Vocab:
    def __init__(self, tokenizer):
        self.index2word = {0: "SOS", 1: "EOS", 2: "UNK", 3: "PAD"}
        self.word2index = {v: k for k, v in self.index2word.items()}
        self.word2count = {}
        self.tokenizer = tokenizer

        self.n_words = 4
      
    def index_words(self, sentence):
        tokenized = self.tokenizer(sentence)
        for word in tokenized:
            self.index_word(word)
        return tokenized

    def index_word(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1
    
    def unindex_words(self, indices):
        return ' '.join([self.index2word[i] for i in indices])

In [13]:
# Lowercase, trim, and remove non-letter characters
def normalize_string(s):
    s = s.lower().strip()
    return s

To read the data file we will split the file into lines, and then split lines into pairs. Define tokenizers for you languages to split sentences on words.

In [166]:
source_tokenizer = tokenize_words
target_tokenizer = nltk.tokenize.WordPunctTokenizer().tokenize

def read_langs(source_lang, source_tokenizer, target_lang, target_tokenizer, input_file):
    corpora = pd.read_csv(input_file, delimiter='\t')
    
    source_vocab = Vocab(source_tokenizer)
    target_vocab = Vocab(target_tokenizer)
    
    source_corpora = []
    target_corpora = []
    for i, row in tqdm(corpora.iterrows()):
        source_sent = row['text'+source_lang]
        target_sent = row['text'+target_lang]
        
        source_tokenized = source_vocab.index_words(source_sent)
        target_tokenized = target_vocab.index_words(target_sent)
        
        source_corpora.append(source_tokenized)
        target_corpora.append(target_tokenized)
    
    return source_vocab, target_vocab, list(zip(source_corpora, target_corpora))

source_vocab, target_vocab, corpora = read_langs(source_lang, source_tokenizer, target_lang, target_tokenizer, file_name)

A Jupyter Widget

### Filtering

Since there are a lot of example sentences and we want to train something quickly, we'll trim the data set to only relatively short and simple sentences. Here the maximum length is 10 words (that includes punctuation) and we're filtering to sentences that translate to the form "I am" or "He is" etc. (accounting for apostrophes being removed).

In [16]:
MAX_LENGTH = 6

corpora_filtered = [(source_sent, target_sent) for source_sent, target_sent in corpora
                   if len(source_sent) <= MAX_LENGTH and len(target_sent) <= MAX_LENGTH]

In [17]:
len(corpora_filtered)

9458

### Turning training data into Tensors
To train we need to turn the sentences into something the neural network can understand, which of course means numbers. Each sentence will be split into words and turned into a Tensor, where each word is replaced with the index (from the Lang indexes made earlier). While creating these tensors we will also append the EOS token to signal that the sentence is over.

A Tensor is a multi-dimensional array of numbers, defined with some type e.g. FloatTensor or LongTensor. In this case we'll be using LongTensor to represent an array of integer indexes.

In [106]:
def indexes_from_sentence(vocab, sentence):
    return [vocab.word2index[word] for word in sentence]

def tensor_from_sentence(lang, sentence):
    indexes = indexes_from_sentence(lang, sentence)
    indexes.append(EOS_token)
    indexes.insert(0, SOS_token)
    if len(indexes) < 8:
        indexes += [PAD_token]*(8-len(indexes))
    tensor = torch.LongTensor(indexes)
    if USE_CUDA: var = tensor.cuda()
    return tensor

def tensors_from_pair(source_sent, target_sent):
    source_tensor = tensor_from_sentence(source_vocab, source_sent).unsqueeze(1)
    target_tensor = tensor_from_sentence(target_vocab, target_sent).unsqueeze(1)
    
    return (source_tensor, target_tensor)

tensors = []
for source_sent, target_sent in corpora_filtered:
    tensors.append(tensors_from_pair(source_sent, target_sent))
    
x, y = zip(*tensors)
x = torch.transpose(torch.cat(x, dim=-1), 1, 0)
y = torch.transpose(torch.cat(y, dim=-1), 1, 0)

## Encoder

The encoder of a seq2seq network is a RNN that outputs some value for every word from the input sentence. For every input word the encoder outputs a vector and a hidden state, and uses the hidden state for the next input word.

In [120]:
class EncoderRNN(nn.Module):
    def __init__(self, vocab_size, hidden_size, n_layers=1):
        super(EncoderRNN, self).__init__()
        
        self.vocab_size = vocab_size
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(vocab_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, num_layers=n_layers, batch_first=True, bidirectional=True)
        
    def forward(self, word_inputs, hidden): # word_inputs: (batch_size, seq_length), h: (h_or_c, layer_n_direction, batch, seq_length)
        # Note: we run this all at once (over the whole input sequence)
        seq_len = len(word_inputs)
        
        # embedded (batch_size, seq_length, hidden_size)
        embedded = self.embedding(word_inputs) 
        # output (batch_size, seq_length, hidden_size*directions)
        # hidden (h: (batch_size, num_layers*directions, hidden_size),
        #         c: (batch_size, num_layers*directions, hidden_size))
        output, hidden = self.lstm(embedded, hidden)
        return output, hidden

    def init_hidden(self, batches):
        hidden = torch.zeros(2, self.n_layers*2, batches, self.hidden_size)
        if USE_CUDA: hidden = hidden.cuda()
        return hidden

## Decoder

In [75]:
class DecoderRNN(nn.Module):
    def __init__(self, vocab_size, hidden_size, n_layers=1):
        super(DecoderRNN, self).__init__()
        
        self.vocab_size = vocab_size
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(vocab_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, num_layers=n_layers, batch_first=True, bidirectional=False)
        
    def forward(self, word_inputs, hidden):
        # Note: we run this one by one
        # embedded (batch_size, 1, hidden_size)
        embedded = self.embedding(word_inputs).unsqueeze_(1)
        output, hidden = self.lstm(embedded, hidden)
        return output, hidden

## Test

To make sure the Encoder and Decoder model are working (and working together) we'll do a quick test with fake word inputs:

In [65]:
encoder_test = EncoderRNN(10, 10, 2)
print(encoder_test)

encoder_hidden = encoder_test.init_hidden()
word_input = torch.LongTensor([[1, 2, 3]])

if USE_CUDA:
    encoder_test.cuda()
    word_input = word_input.cuda()

encoder_outputs, encoder_hidden = encoder_test(word_input, encoder_hidden)

# (batch_size, seq_length, hidden_size)
encoder_outputs.shape

EncoderRNN(
  (embedding): Embedding(10, 10)
  (lstm): LSTM(10, 10, num_layers=2, batch_first=True, bidirectional=True)
)


torch.Size([1, 3, 20])

In [76]:
decoder_test = DecoderRNN(10, 20, 2)
print(decoder_test)

word_inputs = torch.LongTensor([[1, 2, 3]])

decoder_hidden_h = encoder_hidden[0].reshape(2, 1, 20)
decoder_hidden_c = encoder_hidden[1].reshape(2, 1, 20)

if USE_CUDA:
    decoder_test.cuda()
    word_inputs = word_inputs.cuda()

for i in range(3):
    input = word_inputs[:, i]
    decoder_output, decoder_hidden = decoder_test(input, (decoder_hidden_h, decoder_hidden_c))
    decoder_hidden_h, decoder_hidden_c = decoder_hidden
    print(decoder_output.size(), decoder_hidden_h.size(), decoder_hidden_c.size())

DecoderRNN(
  (embedding): Embedding(10, 20)
  (lstm): LSTM(20, 20, num_layers=2, batch_first=True)
)
torch.Size([1, 1, 20]) torch.Size([2, 1, 20]) torch.Size([2, 1, 20])
torch.Size([1, 1, 20]) torch.Size([2, 1, 20]) torch.Size([2, 1, 20])
torch.Size([1, 1, 20]) torch.Size([2, 1, 20]) torch.Size([2, 1, 20])


In [199]:
class Seq2seq(nn.Module):
    def __init__(self, input_vocab_size, output_vocab_size, hidden_size):
        super(Seq2seq, self).__init__()
        
        self.encoder = EncoderRNN(input_vocab_size, int(hidden_size/2), 1)
        self.decoder = DecoderRNN(output_vocab_size, hidden_size, 1)
        
        self.W = nn.Linear(hidden_size, output_vocab_size)
        self.softmax = nn.Softmax()
        
        self.hidden_size = hidden_size
    
    def forward_train(self, x, y):
        batch_size = x.shape[0]
        init_hidden = self.encoder.init_hidden(batch_size)
        encoder_outputs, encoder_hidden = self.encoder(x, init_hidden)
        
        decoder_hidden_h = encoder_hidden[0].reshape(1, batch_size, self.hidden_size)
        decoder_hidden_c = encoder_hidden[1].reshape(1, batch_size, self.hidden_size)
        
        H = []
        for i in range(y.shape[1]):
            input = y[:, i]
            decoder_output, decoder_hidden = self.decoder(input, (decoder_hidden_h, decoder_hidden_c))
            decoder_hidden_h, decoder_hidden_c = decoder_hidden
            # h: (batch_size, vocab_size)
            h = self.W(decoder_output.squeeze(1))
            # h: (batch_size, vocab_size, 1)
            H.append(h.unsqueeze(2))
        
        # H: (batch_size, vocab_size, seq_len)
        return torch.cat(H, dim=2)
    
    def forward(self, x):
        batch_size = x.shape[0]
        init_hidden = self.encoder.init_hidden(batch_size)
        encoder_outputs, encoder_hidden = self.encoder(x, init_hidden)
        
        decoder_hidden_h = encoder_hidden[0].reshape(1, batch_size, self.hidden_size)
        decoder_hidden_c = encoder_hidden[1].reshape(1, batch_size, self.hidden_size)
        
        current_y = SOS_token
        result = [current_y]
        counter = 0
        while current_y != EOS_token and counter < 100:
            input = torch.tensor([current_y])
            decoder_output, decoder_hidden = self.decoder(input, (decoder_hidden_h, decoder_hidden_c))
            decoder_hidden_h, decoder_hidden_c = decoder_hidden
            # h: (vocab_size)
            h = self.W(decoder_output.squeeze(1)).squeeze(0)
            y = self.softmax(h)
            _, current_y = torch.max(y, dim=0)
            current_y = current_y.item()
            result.append(current_y)
            counter += 1
            
        return result

In [219]:
from torch.optim import Adam

model = Seq2seq(source_vocab.n_words, target_vocab.n_words, 60)
optim = Adam(model.parameters(), lr=0.01)

In [175]:
# shuffle training indices
assert x.shape[0] == y.shape[0]
batch_indices = torch.randperm(x.shape[0])
# prepare minibatch generator
def batch_generator(batch_indices, batch_size):
    batches = math.ceil(len(batch_indices)/batch_size)
    for i in range(batches):
        batch_start = i*batch_size
        batch_end = (i+1)*batch_size
        if batch_end > len(batch_indices):
            yield batch_indices[batch_start:]
        else:
            yield batch_indices[batch_start:batch_end]

In [ ]:
BATCH_SIZE = 100
# cross_entropy = nn.CrossEntropyLoss()
nll = nn.NLLLoss()
softmax = nn.Softmax(dim=1)
for epoch in range(100):
    total_loss = 0
    for batch in tqdm(batch_generator(batch_indices, BATCH_SIZE), 
                      desc='Training epoch {}'.format(epoch+1), total=95):
        x_train = x[batch, :] 
        y_train = y[batch, :]
        H = model.forward_train(x_train, y_train)
#         loss = cross_entropy(H, y_train)
        H = softmax(H)
        loss = nll(H, y_train)
        print(H[0, :, 2], y[0])
        break
        
        optim.zero_grad()
        loss.backward()
        optim.step()
        
        total_loss += loss.item()
    
    print('Epoch {} training finished, loss: {}'.format(epoch+1, total_loss/95))

In [177]:
def generate(x, model):
    y_gen = model(x)
    return y_gen

x_0, y_0 = x[0], y[0]

In [178]:
x_0

tensor([  0,  18,  19,  20,  21,  11,   1,   3])

In [179]:
corpora_filtered[0]

(['Вона', 'приймає', 'душ', 'щоранку', '.'],
 ['Sie', 'duscht', 'jeden', 'Morgen', '.'])

In [237]:
source_vocab.unindex_words(x_train[0].numpy())

'SOS Двері відкриті . EOS PAD PAD PAD'

In [238]:
target_vocab.unindex_words(y_train[0].numpy())

'SOS Die Tür ist auf . EOS PAD'

In [225]:
y_0_gen = model(x_0.unsqueeze(0))

/Users/anatolii.stehnii/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:51: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [226]:
target_vocab.unindex_words(y_0_gen)

'SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS SOS'

In [235]:
x_train[0]

tensor([    0,  2617,  2618,    11,     1,     3,     3,     3])

In [236]:
y_train[0]

tensor([   0,  157,  755,   29,  317,   13,    1,    3])